In [3]:
from models import ResNetModel
from datasets import Caltech101DataModule, Caltech256DataModule
from lightning import Trainer
import torch

# Load datasets
caltech101 = Caltech101DataModule()
caltech256 = Caltech256DataModule()

# Load models
caltech101_model = ResNetModel.load_from_checkpoint(
    "checkpoints/resnet50-caltech101-min-val-loss.ckpt"
)
caltech101_model.eval()
caltech256_model = ResNetModel.load_from_checkpoint(
    "checkpoints/resnet50-caltech256-min-val-loss.ckpt"
)
caltech256_model.eval()

# Test foreign datasets
trainer = Trainer(logger=False, enable_checkpointing=False)
outputs_caltech101_model_caltech256 = trainer.predict(
    caltech101_model, datamodule=caltech256
)
outputs_caltech256_model_caltech101 = trainer.predict(
    caltech256_model, datamodule=caltech101
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 48/48 [00:11<00:00,  4.18it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 14/14 [00:02<00:00,  5.83it/s]


In [ ]:
import pandas as pd

# Get predictions
predictions_caltech101_model_caltech256 = torch.cat(
    outputs_caltech101_model_caltech256
).argmax(dim=1)
predictions_caltech256_model_caltech101 = torch.cat(
    outputs_caltech256_model_caltech101
).argmax(dim=1)

# Get targets and labels
caltech101_targets = torch.cat([label for _, label in caltech101.predict_dataloader()])
caltech256_targets = torch.cat([label for _, label in caltech256.predict_dataloader()])

# Save dataframes
df_caltech101 = pd.DataFrame(
    {
        "targets": caltech101_targets.numpy(),
    }
)
df_caltech101.to_csv(
    "output/caltech101.csv",
    index=False,
)
df_caltech256 = pd.DataFrame(
    {
        "targets": caltech256_targets.numpy(),
    }
)
df_caltech256.to_csv(
    "output/caltech256.csv",
    index=False,
)
df_caltech101_model_caltech256 = pd.DataFrame(
    {
        "predictions": predictions_caltech101_model_caltech256.numpy(),
    }
)
df_caltech101_model_caltech256.to_csv(
    "output/caltech101_model_caltech256.csv",
    index=False,
)
df_caltech256_model_caltech101 = pd.DataFrame(
    {
        "predictions": predictions_caltech256_model_caltech101.numpy(),
    }
)
df_caltech256_model_caltech101.to_csv(
    "output/caltech256_model_caltech101.csv",
    index=False,
)